## Decision tree

In [163]:
import pandas as pd

In [164]:
df= pd.read_csv("/Users/tanjamarkotic/Desktop/DMML2019_Team_Swatch/data/DataWithLocationCleaned.csv.gz")

In [165]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [166]:
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Year,Month,position
0,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Full,133,2019,6,"(48.905266, -95.314404)"
1,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Truck Containers Empty,298,2019,6,"(48.905266, -95.314404)"
2,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicles,10383,2019,6,"(48.905266, -95.314404)"
3,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Personal Vehicle Passengers,19459,2019,6,"(48.905266, -95.314404)"
4,Warroad,Minnesota,3423,US-Canada Border,2019-06-01,Pedestrians,2,2019,6,"(48.905266, -95.314404)"


In [167]:
# checking types
df.dtypes

Port Name    object
State        object
Port Code     int64
Border       object
Date         object
Measure      object
Value         int64
Year          int64
Month         int64
position     object
dtype: object

In [168]:
# correct types and add columns longitude/latitude cleaned
df['Date'] = pd.to_datetime(df['Date'])
df['position']=df['position'].str.replace(" ", "").str.strip('(').str.strip(')')
df['latitude']=df['position'].str.split(',').str[0].str.split('.').str[0]
df['longitude']=df['position'].str.split(',').str[1].str.split('.').str[0]
df['latitude'] = df['latitude'].replace(['InternationalFalls'], 48)
df['latitude'] = df['latitude'].replace(['PleasantCamp'], 59)
df['latitude'] = df['latitude'].replace("\'48", '48')
df['latitude'] = df['latitude'].replace("\'44", '44')
df['longitude'] = df['longitude'].replace('KoochichingCounty', '-93')
df['longitude'] = df['longitude'].replace('Skagway', '-136')
df['longitude'] = df['longitude'].replace('\'-116', '-116')
df['longitude'] = df['longitude'].replace('\'-66', '-66')
df['latitude'] = pd.to_numeric(df['latitude'])
df['longitude'] = pd.to_numeric(df['longitude'])
df.dtypes


Port Name            object
State                object
Port Code             int64
Border               object
Date         datetime64[ns]
Measure              object
Value                 int64
Year                  int64
Month                 int64
position             object
latitude              int64
longitude             int64
dtype: object

In [169]:
df = df[df['Measure'].isin(['Bus Passengers','Pedestrians','Personal Vehicle Passengers','Train Passengers'])]

In [170]:
df.groupby(['Port Name']).sum().sort_values(['Value'],
                                            ascending=False).head()



,Port Code,Value,Year,Month,latitude,longitude
Port Name,,,,,,
San Ysidro,2736872,820753587,2193570,7033,34976,-127881
El Paso,2709456,813405376,2264184,7260,34968,-119568
Laredo,2518272,435119695,2193570,7033,29511,-108207
Hidalgo,2519365,406092835,2193570,7033,28418,-107114
Calexico,2648174,401463611,2122956,6806,33856,-121670


In [171]:
df.shape

(116131, 12)

In [172]:
 df.isnull().sum()

Port Name    0
State        0
Port Code    0
Border       0
Date         0
Measure      0
Value        0
Year         0
Month        0
position     0
latitude     0
longitude    0
dtype: int64

In [173]:
# check values...
df['Value'].describe()

count    1.161310e+05
mean     5.778344e+04
std      2.366958e+05
min      0.000000e+00
25%      0.000000e+00
50%      1.560000e+02
75%      6.943000e+03
max      4.447374e+06
Name: Value, dtype: float64

In [174]:
df['Value category'] = pd.cut(df.Value, bins=10, right=False)

In [175]:
# check categories...
df.sort_values(['Value'],ascending=False).head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Year,Month,position,latitude,longitude,Value category
271924,El Paso,Texas,2402,US-Mexico Border,2001-03-01,Personal Vehicle Passengers,4447374,2001,3,"31.7600372,-106.487287",31,-106,"[4002636.6, 4451821.374)"
280759,El Paso,Texas,2402,US-Mexico Border,2000-08-01,Personal Vehicle Passengers,4291774,2000,8,"31.7600372,-106.487287",31,-106,"[4002636.6, 4451821.374)"
273470,El Paso,Texas,2402,US-Mexico Border,2001-02-01,Personal Vehicle Passengers,4281175,2001,2,"31.7600372,-106.487287",31,-106,"[4002636.6, 4451821.374)"
270913,El Paso,Texas,2402,US-Mexico Border,2001-04-01,Personal Vehicle Passengers,4252128,2001,4,"31.7600372,-106.487287",31,-106,"[4002636.6, 4451821.374)"
282706,El Paso,Texas,2402,US-Mexico Border,2000-06-01,Personal Vehicle Passengers,4235244,2000,6,"31.7600372,-106.487287",31,-106,"[4002636.6, 4451821.374)"


In [195]:
df['Value in millions'] = (df['Value']/1000000).apply(lambda x: '{:,.2f}'.format(x))

In [196]:
#features but possibly add lat, long, measure
X = df[["Year","Month", "Port Code"]]
y = df["Value in millions"]
X.shape

(116131, 3)

In [212]:
y = y.astype('float')


In [213]:
one_hot = OneHotEncoder(categories="auto")
cat_to_onehot = one_hot.fit_transform(df[["Border"]]).toarray()
cat_to_onehot = pd.DataFrame(cat_to_onehot)
cat_to_onehot.shape

(116131, 2)

In [214]:
X = pd.concat((X, cat_to_onehot), axis=1)

X.shape

(116131, 15)

In [221]:
X

,Year,Month,Port Code,0,1,0,1,0,1,0,1,0,1,0,1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
3,2019.0,6.0,3423.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,2019.0,6.0,3423.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0
6,2019.0,6.0,3423.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0


In [215]:
X.isnull().sum()

Year         77696
Month        77696
Port Code    77696
0            77696
1            77696
0            77696
1            77696
0            77696
1            77696
0            77696
1            77696
0                0
1                0
0                0
1                0
dtype: int64

In [216]:
X=X.astype('float')


In [217]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)

In [218]:
clf = DecisionTreeClassifier(criterion='entropy')

In [219]:
clf.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [220]:
# test accuracy
clf.score(X_test,y_test)

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
# depth of the decision tree
clf.get_depth()